# Gemini GMOS Photometry reduction using DRAGONS Python API
***
## Public archival data from program GS2018A-Q-207 - NGC5018
#### adapted from https://gmosimg-drtutorial.readthedocs.io/en/v2.1.1/03_api_reduction.html
#### don't forget to `conda install -n dragons nb_conda_kernels ipykernel` to run this notebook on the DRAGONS env
***

**Importing Python Libraries** (you'll probably have to install the `wget` and `ipympl` libraries)

In [ ]:
from __future__ import print_function

import glob
import wget

from gempy.adlibrary import dataselect
from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce
from gempy.utils import logutils

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm

#%matplotlib widget
%matplotlib inline

**Downloading the data**

In [ ]:
# Science data from GS2018A-Q-207
wget.download("http://archive.gemini.edu/file/S20180419S0098.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0099.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0100.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0101.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0102.fits")
# Twilight flats 
wget.download("http://archive.gemini.edu/file/S20180419S0207.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0208.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0209.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0210.fits")
wget.download("http://archive.gemini.edu/file/S20180419S0211.fits")
# Biases 
wget.download("http://archive.gemini.edu/file/S20180423S0050.fits")
wget.download("http://archive.gemini.edu/file/S20180423S0051.fits")
wget.download("http://archive.gemini.edu/file/S20180423S0052.fits")
wget.download("http://archive.gemini.edu/file/S20180423S0053.fits")
wget.download("http://archive.gemini.edu/file/S20180423S0054.fits")

**Create and move data to raw/ directory**

In [ ]:
!mkdir raw/
!mv S2018*.fits raw/

**Setting up the DRAGONS logger**

In [ ]:
logutils.config(file_name='gmos_data_reduction.log')

**Setting up the Calibration Service**

First, check that you have already a `rsys.cfg` file inside the `~/.geminidr/`. It should contain:

```[calibs]
standalone = True
database_dir = /path_to_this_folder/```

In [ ]:
caldb = cal_service.CalibrationService()
caldb.config()
caldb.init()

cal_service.set_calservice()

**Create a list of all the FITS files in the directory**

In [ ]:
all_files = glob.glob('raw/S2018*[0-9].fits')
all_files.sort()
#all_files

**List of Biases, Flats, and Science frames**

In [ ]:
list_biases  = dataselect.select_data(all_files,['BIAS'],[])
list_flats   = dataselect.select_data(all_files,['FLAT'],[],
                                      dataselect.expr_parser('filter_name=="g"'))
list_science = dataselect.select_data(all_files,[],['CAL'],
                                      dataselect.expr_parser('(observation_class=="science" and filter_name=="g")'))

**Make master Bias/Flat and add to the calibration database**

In [ ]:
reduce_bias = Reduce()
reduce_bias.files.extend(list_biases)
reduce_bias.runr()

caldb.add_cal(reduce_bias.output_filenames[0])

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_flats)
reduce_flats.runr()

caldb.add_cal(reduce_flats.output_filenames[0])

**Reduce Science Images**

In [ ]:
reduce_science = Reduce()
reduce_science.files.extend(list_science)
reduce_science.runr()

**Display the Stacked Image**

In [ ]:
image_file = "S20180419S0098_stack.fits"
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (15,15))
plt.imshow(image_data,cmap='gray',norm=LogNorm(vmin=3000, vmax=60000))
plt.xlim(900,2500)
plt.ylim(1600,600)
plt.show()

**Optional: remove duplicate calibrations and remove raw data (uncomment lines before running)**

In [ ]:
#!rm -rf *_bias.fits *_flat.fits
#!rm -rf raw/